我认为擅长讲故事的方法应该有上面提到的四个部分组成，而且它们之间应该做到泾渭分明，而不是混杂在一起。但这还不是，它们的排列顺序也有要求：首先处理输入；接着实现核心功能；然后处理输出；如有需要，最后处理异常。

无论用哪种方式，`switch`之类的语句都杂糅了太多方法逻辑，使得程序更难调试，使得对象类型绑架了程序逻辑。身为自信优雅的程序员，我们只需要让鸭子叫，然后可以放心的走开。这一位这首先的找出所需的消息和校色，然后确保只有那些真正会叫的“鸭子”才嫩刚进入我们的方法逻辑。

到目前为止，我们见过了直接把值本省作为输入的方式，我们把这种输入称为“简单输入”或“直接输入”。但`Time.now`却是一个间接输入。首先我们引用了`Time`类，然后把`#now`消息发送给`Time`。我们想要的其实是方法`now`的返回值，而不是`Time`常量本身。任何我时候为了得到方法返回值，而向其他对象发送消息，其实就在使用间接输入。

In [2]:
require 'yaml'

def format_time
  user = ENV['USER']
  prefs = YAML.load_file('/home/#{user}/time-prefs.yml')
  format = prefs.fetch('format') {'%D %r'}
  Time.now.strftime(format)
end

:format_time

在此示例中，两个“间接输入”协同工作去得到最终值，而其中的一个间接输入的值之后为了获取另外一个“间接输入”的值。这通常都是软件中`bug`的源头之一。